In [1]:
# Importing necessary libraries and modules


from sqlalchemy import create_engine
import datetime as dt
import pandas as pd
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler

In [2]:
# pandas options 

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [3]:
#function for detecting and removing outliers

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [4]:
df_ = pd.read_excel(r"C:\Users\ExtraBT\Downloads\online_retail_II.xlsx", sheet_name="Year 2010-2011")
df=df_.copy()

In [5]:
# Veri Ön İşleme

df.describe().T
df.isnull().sum()
df.dropna(inplace=True) #remove missing values
df.isnull().values.any() 
df = df[~df["Invoice"].str.contains("C", na=False)] # remove cancelled orders
df = df[df["Quantity"] > 0] # remove negative quantities


In [6]:
# filtering UK customers

df=df[df['Country']=='United Kingdom']
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [7]:
 # Remove outliers
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,354345.0000,10.6769,22.0755,1.0000,2.0000,4.0000,12.0000,248.5000
Price,354345.0000,2.8150,2.9206,0.0000,1.2500,1.9500,3.7500,31.5600
Customer ID,354345.0000,15552.4362,1594.5460,12346.0000,14194.0000,15522.0000,16931.0000,18287.0000


In [8]:
# create 'Total Price' variable


df["TotalPrice"] = df["Quantity"] * df["Price"]

df['TotalPrice'].head()

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0000,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom,15.3000
1,536365,71053,WHITE METAL LANTERN,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0000,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom,22.0000
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0000,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom,20.3400


In [9]:
#Date of analysis 

today_date = dt.datetime(2011, 12, 11) 

 

In [10]:
# Creating recency,frequency and monetary variables for each customer 

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days, #recency
                                                         lambda date: (today_date - date.min()).days], #T
                                         'Invoice': lambda num: num.nunique(), # frequency
                                         'TotalPrice': lambda TotalPrice: TotalPrice.sum()}) #monetary

cltv_df.columns = cltv_df.columns.droplevel(0)

cltv_df.columns = ['recency', 'T', 'frequency', 'monetary'] #Assigning variable names 

cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,258.4400
12747.0000,366,370,11,4196.0100
12748.0000,372,374,210,32303.9100
12749.0000,209,214,5,4072.4400
12820.0000,323,327,4,942.3400


In [11]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"] # average monetary values for each customer 

cltv_df=cltv_df[cltv_df['monetary']>0] # remove negative monetary values

cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0000,0,326,1,258.4400
12747.0000,366,370,11,381.4555
12748.0000,372,374,210,153.8281
12749.0000,209,214,5,814.4880
12820.0000,323,327,4,235.5850


In [12]:
# converting recency and T into weekly values 

cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

In [13]:
# Filtering customers with more than one transaction
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [14]:
#BGNBD Model

bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'], cltv_df['recency'], cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2570 subjects, a: 0.12, alpha: 11.66, b: 2.51, r: 2.21>

In [15]:
cltv_df.isnull().any()

recency      False
T            False
frequency    False
monetary     False
dtype: bool

In [16]:
cltv_df['recency']

Customer ID
12747.0000   52.2857
12748.0000   53.1429
12749.0000   29.8571
12820.0000   46.1429
12822.0000    2.2857
               ...  
18272.0000   34.8571
18273.0000   36.4286
18282.0000   16.8571
18283.0000   47.5714
18287.0000   22.5714
Name: recency, Length: 2570, dtype: float64

In [17]:
# 3. GAMMA-GAMMA Model

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 2570 subjects, p: 3.82, q: 0.35, v: 3.75>

In [18]:
#Expected average profit

cltv_df['expected_average_profit']=ggf.conditional_expected_average_profit(cltv_df['frequency'], cltv_df['monetary'])
cltv_df.head()

,recency,T,frequency,monetary,expected_average_profit
Customer ID,,,,,
12747.0000,52.2857,52.8571,11,381.4555,387.8228
12748.0000,53.1429,53.4286,210,153.8281,153.9713
12749.0000,29.8571,30.5714,5,814.4880,844.0947
12820.0000,46.1429,46.7143,4,235.5850,247.0809
12822.0000,2.2857,12.5714,2,474.4400,520.8282


In [19]:
# 4. 6 month CLTV calculation with BG-NBD and GG model 

cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 months
                                   freq="W",  # T as weekly
                                   discount_rate=0.01)


cltv = cltv.reset_index()
cltv_final = cltv_df.merge(cltv, on="Customer ID", how="left")

cltv_final.sort_values(by="clv", ascending=False).head(10)
cltv_final.sort_values(by="clv", ascending=False).tail(10)
cltv_final.describe().T

cltv_final.head()

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv
0,12747.0000,52.2857,52.8571,11,381.4555,387.8228,1937.0461
1,12748.0000,53.1429,53.4286,210,153.8281,153.9713,12365.7962
2,12749.0000,29.8571,30.5714,5,814.4880,844.0947,3446.0104
3,12820.0000,46.1429,46.7143,4,235.5850,247.0809,631.9393
4,12822.0000,2.2857,12.5714,2,474.4400,520.8282,1612.0967


In [20]:
# Calculating 1 month CLTV

cltv_1 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,  # 1 month
                                   freq="W",  # T as weekly
                                   discount_rate=0.01)

cltv_1=cltv_df.merge(cltv_1,on='Customer ID',how='left')

cltv_1.reset_index(inplace=True)

cltv_1.head(10)

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv
0,12747.0000,52.2857,52.8571,11,381.4555,387.8228,336.7788
1,12748.0000,53.1429,53.4286,210,153.8281,153.9713,2148.3757
2,12749.0000,29.8571,30.5714,5,814.4880,844.0947,604.0710
3,12820.0000,46.1429,46.7143,4,235.5850,247.0809,110.1249
4,12822.0000,2.2857,12.5714,2,474.4400,520.8282,286.9223
5,12823.0000,31.5714,42.4286,5,351.9000,365.1326,191.1237
6,12826.0000,51.7143,52.2857,7,210.6743,216.4966,131.6694
7,12827.0000,5.4286,6.4286,3,143.3833,153.3719,181.1336
8,12828.0000,18.1429,18.7143,6,169.7850,175.4071,198.6958
9,12829.0000,3.2857,51.5714,2,138.1100,153.0668,2.8055


In [24]:
#Calculating 12 Month CLTV

cltv_12 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=1,  # 12  month
                                   freq="W",  # T as weekly
                                   discount_rate=0.01)

cltv_12=cltv_df.merge(cltv_12,on='Customer ID',how='left')

cltv_12.reset_index(inplace=True)

cltv_12.head()

,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv
0,12747.0000,52.2857,52.8571,11,381.4555,387.8228,336.7788
1,12748.0000,53.1429,53.4286,210,153.8281,153.9713,2148.3757
2,12749.0000,29.8571,30.5714,5,814.4880,844.0947,604.0710
3,12820.0000,46.1429,46.7143,4,235.5850,247.0809,110.1249
4,12822.0000,2.2857,12.5714,2,474.4400,520.8282,286.9223


In [22]:
#Comparing 1 month and 12 month CLTV Values 

cltv_1.sort_values(by="clv", ascending=False).head(10)
cltv_12.sort_values(by="clv", ascending=False).head(10)




,Customer ID,recency,T,frequency,monetary,expected_average_profit,clv
2486,18102.0000,52.2857,52.5714,60,3584.8878,3595.1926,14884.9750
589,14096.0000,13.8571,14.5714,17,3159.0771,3191.3867,9855.8799
2184,17450.0000,51.2857,52.5714,46,2629.5299,2639.4193,8434.7647
2213,17511.0000,52.8571,53.4286,31,2921.9519,2938.2745,6394.3243
1804,16684.0000,50.4286,51.2857,28,2120.0470,2133.2036,4361.0540
587,14088.0000,44.5714,46.1429,13,3859.6015,3911.3188,4355.4853
406,13694.0000,52.7143,53.4286,50,1267.3626,1271.7854,4354.4685
1173,15311.0000,53.2857,53.4286,91,667.5968,668.8945,4098.8694
133,13089.0000,52.2857,52.8571,97,605.1866,606.2937,3984.0515
1485,16000.0000,0.0000,0.4286,3,2055.7867,2181.3236,3843.9795


In [23]:
#  Segmentation
##############################################################

cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])

cltv_final.head()

cltv_final.groupby("segment").agg( {"count", "mean", "sum"})


Customer ID                     recency                        T               frequency                  monetary                expected_average_profit                         clv                
                  sum       mean count        sum    mean count        sum    mean count       sum    mean count         sum     mean count                     sum     mean count          sum      mean count
segment                                                                                                                                                                                                        
D       10099073.0000 15706.1788   643 14189.1429 22.0671   643 26047.2857 40.5090   643      1973  3.0684   643 114084.2713 177.4250   643             123598.0776 192.2210   643  173267.3118  269.4670   643
C        9964742.0000 15521.4050   642 19797.0000 30.8364   642 24502.2857 38.1656   642      2562  3.9907   642 167362.5081 260.6893   642             178520.7809 278.0698   642  456348.8182  710.8237   642
B       10012848.0000 15596.3364   642 19156.5714 29.8389   642 22545.1429 35.1170   642      3502  5.4548   642 225975.5615 351.9869   642             237858.1615 370.4956   642  816208.5590 1271.3529   642
A        9896309.0000 15390.8383   643 20229.1429 31.4606   643 22195.8571 34.5192   643      7261 11.2924   643 376992.3231 586.3022   643             391391.9252 608.6966   643 2447482.3888 3806.3490   643